In [5]:
from selenium import webdriver
options = webdriver.ChromeOptions() 
options.add_argument('--headless') 
options.add_argument('--no-sandbox') 
options.add_argument('--disable-dev-shm-usage') 
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import time
import pandas as pd
import re
pd.set_option('display.max_colwidth', None)

In [56]:
TARGET_URL = 'https://news.naver.com/main/ranking/popularMemo.nhn?date=20210308'

TITLE_OF_ARTICLE = []
OFFICE_OF_ARTICLE = []
URL_OF_ARTICLE = []
DATE_OF_ARTICLE = []
CONTENT_OF_ARTICLE = []
COMMENT_OF_ARTICLE = []

In [75]:
def get_link_from_news_title(URL):
    req = urllib.request.Request(URL, headers={'User-Agent': 'Mozilla/5.0'})
    source_code_from_url = urllib.request.urlopen(req)
    soup = BeautifulSoup(source_code_from_url, 'lxml', from_encoding='utf-8')
    
    for title in soup.select('div.list_content'):
        url_link = title.select('a')
        article_URL = 'https://news.naver.com' + url_link[0]['href']
        URL_OF_ARTICLE.append(article_URL)

        position = article_URL.index('read.nhn?')
        comment_URL = article_URL[:position+9] + 'm_view=1&includeAllCount=true&' + article_URL[position+9:] 

        get_text(article_URL) 
        get_comment(comment_URL)

In [76]:
def get_text(article_URL):
    req = urllib.request.Request(article_URL, headers={'User-Agent': 'Mozilla/5.0'})
    source_code_from_url = urllib.request.urlopen(req)
    soup = BeautifulSoup(source_code_from_url, 'lxml', from_encoding='utf-8')

    #office_link = soup.select('head > meta:nth-of-type(6)')
    office_link = soup.select('div.press_logo > a > img')
    OFFICE_OF_ARTICLE.append(office_link[0]['title'])

    title_link = soup.select('div.article_info > h3')
    TITLE_OF_ARTICLE.append(title_link[0].string)

    date_of_article = soup.select('span.t11')
    DATE_OF_ARTICLE.append(date_of_article[0].string)

    string_item = '' #빈 문자열 만드는 것
    for item in soup.select('div._article_body_contents'):  
        string = str(item.find_all(text=True))
        string_item += string 
    CONTENT_OF_ARTICLE.append(string_item) 

In [1]:
def get_comment(comment_URL):
    driver = webdriver.Chrome(executable_path=r'C:/chromedriver/chromedriver.exe')
    driver.implicitly_wait(5)
    driver.get(comment_URL) #URL 창을 여는 것

    
    contents = driver.find_elements_by_css_selector('span.u_cbox_contents')
    comments = []
    for comment in contents:
        comments.append(comment.text) #BS4에서는 .string이었고 셀 에서는 .text

    COMMENT_OF_ARTICLE.append(comments)

In [81]:
start = time.time()
get_link_from_news_title(TARGET_URL)
print("time :", time.time() - start)

df = pd.DataFrame(list(zip(DATE_OF_ARTICLE, TITLE_OF_ARTICLE, OFFICE_OF_ARTICLE, CONTENT_OF_ARTICLE, URL_OF_ARTICLE, COMMENT_OF_ARTICLE)), columns =['Date', 'Title', 'Office', 'Content', 'URL', 'Comment']) 

df.to_csv("4주차실습(네이버).csv",index=True, encoding='utf-8-sig')

time : 1640.3320517539978


# ===========================================================

In [6]:
TARGET_URL = 'https://www.chosun.com/search/query=%EC%9D%B8%EA%B6%8C&siteid=www&sort=1&date_period=1w&writer=&field=&emd_word=&expt_word=&opt_chk=false/'


TITLE_OF_ARTICLE = []
DATE_OF_ARTICLE = []
CONTENT_OF_ARTICLE = []
URL_OF_ARTICLE = []

In [2]:
def get_link_from_news_title(URL): 
    driver = webdriver.Chrome(executable_path=r'C:/chromedriver/chromedriver.exe')
    driver.implicitly_wait(5)
    driver.get(URL)

    while True: # 반복문 break를 만나기 전까지 무한히 반복한다.
        try: # 더보기 버튼을 더 이상 누를수 없을 때 까지 반복한다.
            more_button = driver.find_element_by_xpath('//*[@id="load-more-stories"]') # 더보기 버튼 위치
            more_button.click() # 더보기 버튼 클릭
            time.sleep(1) # 페이지가 로딩되는데 시간이 필요하므로 1초의 텀을 준다.
        except: # 더보기 버튼을 더 이상 누를수 없을 때 실행된다.
            break # 반복문 종료

    soup = BeautifulSoup(driver.page_source, 'html.parser') # soup로 html 소스를 가져온다.
    
    for title in soup.select('div.story-card__headline-container.\|.box--margin-bottom-xs > h5'): # 기사들의 URL이 있는 링크에 접근해서 하나씩 title로 넘겨주는 반복문...h3이 나올수도 있음
        title_link = title.select('span')
        TITLE_OF_ARTICLE.append(title_link[0].get_text())
        #TITLE_OF_ARTICLE.append(title.span.text) 

        article_link = title.select('a') # URL이 위치하는 태그를 가리킨다.
        article_URL = 'https://www.chosun.com' + article_link[0]['href'] # URL 리스트에 저장한다.
        
        get_text(article_URL) # URL을 get_text 함수에 넘겨준다.

In [8]:
def get_text(URL):
    driver = webdriver.Chrome(executable_path=r'C:/chromedriver/chromedriver.exe')
    driver.implicitly_wait(5)
    driver.get(URL)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    date_of_article = soup.select('div.article-dateline.\|.flex.flex--justify-space-between.flex--align-items-center.box--border.box--border-grey-40.box--border-horizontal.box--border-horizontal-bottom.box--pad-bottom-sm > span')
    DATE_OF_ARTICLE.append(date_of_article[0].string)

    content_of_article = soup.select('section.article-body')
    string_item = str(content_of_article[0].find_all(text=True))
    CONTENT_OF_ARTICLE.append(string_item) 

In [9]:
get_link_from_news_title(TARGET_URL)

df = pd.DataFrame(list(zip(TITLE_OF_ARTICLE, CONTENT_OF_ARTICLE, DATE_OF_ARTICLE)), columns =['Title', 'Content', 'Date']) 

df.to_csv("4주차실습(조선일보).csv",index=True, encoding='utf-8-sig')